In [2]:
import tensorflow as tf
import numpy as np
import pickle
import t3f

In [3]:
with open('weights_of_deepspeech_vanilla.p', 'rb') as fp:
    weights = pickle.load(fp)

In [10]:
for key in weights.keys():
    print(key, ' => ', weights[key].shape)

layer_1/weights  =>  (494, 2048)
layer_2/weights  =>  (2048, 2048)
layer_3/weights  =>  (2048, 2048)
layer_5/weights  =>  (2048, 2048)
layer_6/weights  =>  (2048, 29)


In [40]:
def get_error(tt_input_shape, tt_output_shape, tt_rank, weight):
    weight_input_shape, weight_output_shape = weight.shape
    assert(np.prod(tt_input_shape) == weight_input_shape)
    assert(np.prod(tt_output_shape) == weight_output_shape)
    assert(len(tt_input_shape) == len(tt_output_shape))

    # obtain cores
    tt_weights = t3f.to_tt_matrix(weight, shape=[tt_input_shape, tt_output_shape],
                                   max_tt_rank=tt_rank)
    
    with tf.Session() as sess:
        reconstructed_tensor = sess.run(t3f.full(tt_weights))

    error = np.linalg.norm(reconstructed_tensor - weight)
    initial_norm = np.linalg.norm(weight)
    #with tf.Session() as sess:
    #    cores = sess.run(tt_weights.tt_cores)
    #    ranks = sess.run(tt_ranks)
    #    reconstructed_matrix = t3f.full(tt_weights)
    return error, error / initial_norm

In [41]:
error, rel_error = get_error([8, 8, 8, 4], [8, 8, 8, 4], 16, weights['layer_2/weights'])
print(f'Reconstruction error: {error}, {rel_error}')

Reconstruction error: 94.52107238769531, 0.980887234210968
